# Prepare the dataset

In [1]:
from datasets import load_dataset

dataset = load_dataset('csv', data_files={'train':'train.csv', 'validation':'validation.csv', 'test':'test.csv'})

Found cached dataset csv (C:/Users/Omar/.cache/huggingface/datasets/csv/default-80bb726c2d57ffc1/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
dataset["train"][0]

{'text': 'يعزا تراجع إيرادات ياهو بشكل كبير إلى الانخفاض الحاد في نشاط الإعلانات المرئية الرقمية وتساعد فلاري مطوري التطبيقات على تحليل البيانات المتعلقة بمستخدميهم وتوفير إعلانات تناسب أذواقهم. واشترى بعض منافسي ياهو شركات تعمل في تكنولوجيا إعلانات الهواتف المحمولة في محاولة لجذب المسوقين إلى تطبيقاتها وتحقيق إيرادات من الإعلانات على تطبيقات شركات أخرى. ولم تكشف الشركتان عن البنود المالية لصفقة الاستحواذ. لكن بعض التقارير أشارت إلى أن ياهو دفعت ما بين 200 و300 مليون دولار للاستحواذ على فلاري، لتكون بذلك واحدة من أكبر عمليات الاستحواذ لياهو منذ أن استحوذت على منصة تدوين "تومبير" عام 2012. مواضيع قد تهمك نهاية تعزيز الإيرادات وتأتي صفقة ياهو للاستحواذ على فلاري بعد أيام فقط من إعلان تراجع أرباحها بواقع 18 في المئة إلى 270 مليون دولار خلال الأشهر الثلاثة حتى نهاية يونيو/حزيران الماضي. وتراجعت الإيرادات أيضا في شركة الانترنت العملاقة بواقع ثلاثة في المئة إلى 1.08 مليار دولار. ويعزا هذا التراجع بشكل كبير إلى الانخفاض الحاد في نشاط الإعلانات المرئية الرقمية، التي تراجعت بواقع ثمانية في المئ

In [21]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from arabert.preprocess import ArabertPreprocessor
from transformers import AutoTokenizer
from transformers import AutoModel

In [4]:
model_name = "aubmindlab/bert-base-arabertv2"
preprocessor = ArabertPreprocessor(model_name=model_name)

[2023-07-02 22:10:11,702 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


# Preprocess the dataset

In [7]:
def preprocess(text):
  return preprocessor.preprocess(text)


import re

def clean(text):
  temp = preprocessor.unpreprocess(text)
  temp = re.sub('\[CLS\]|\[PAD\]|\[SEP\]', '', temp)
  return re.sub('\s+$|^\s', '', temp)

# Load the tokenizer

In [8]:
tokenizer = AutoTokenizer.from_pretrained("./AraT5")
model = AutoModelForSeq2SeqLM.from_pretrained("./AraT5")

# Encode the dataset with the tokenizer

In [9]:
def tokenize_function(examples):
  inputs = examples['text']
  model_inputs = tokenizer(inputs, padding='max_length', truncation=True)

  labels = tokenizer(examples['summary'], padding='max_length', max_length=128, truncation=True, return_tensors="pt")

  model_inputs['labels'] = labels['input_ids']
  return model_inputs


In [10]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

small_train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(1000))
small_test_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(1000))

Map:   0%|          | 0/60878 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/7610 [00:00<?, ? examples/s]

Map:   0%|          | 0/7610 [00:00<?, ? examples/s]

In [11]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Load metrics

In [22]:
import evaluate
import numpy as np

rouge = evaluate.load('rouge')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

ImportError: To be able to use evaluate-metric/rouge, you need to install the following dependencies['absl', 'rouge_score'] using 'pip install # Here to have a nice missing dependency error message early on rouge_score' for instance'

# Train the model

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="ouput-model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset["train"],
    eval_dataset=small_test_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()